<a href="https://colab.research.google.com/github/bjelikj/DT2119/blob/master/Labs/Lab3/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
!rm lab3_proto.py
!rm lab3_tools.py
!rm lab1_proto.py
!rm lab3_example.npz
!rm lab2_models_all.npz
!ls

rm: cannot remove 'lab1_proto.py': No such file or directory
rm: cannot remove 'lab3_example.npz': No such file or directory
rm: cannot remove 'lab2_models_all.npz': No such file or directory
drive  __pycache__  sample_data


In [22]:
from google.colab import files

"""
Necesarry files:
- lab3_proto.py
- lab3_tools.py
- lab1_proto.py
- lab1_tools.py

If not uploaded to Google Drive:
- lab2_models_all.npz
- lab3_example.npz
"""

files.upload()

Saving lab1_tools.py to lab1_tools.py


{'lab1_tools.py': b'import numpy as np\r\n# DT2119, Lab 1 Feature Extraction\r\n# - Functions given by the exercise -------------------------------------------- \r\n\r\ndef tidigit2labels(tidigitsarray):\r\n    """\r\n    Return a list of labels including gender, speaker, digit and repetition information for each\r\n    utterance in tidigitsarray. Useful for plots.\r\n    """\r\n    labels = []\r\n    nex = len(tidigitsarray)\r\n    for ex in range(nex):\r\n        labels.append(tidigitsarray[ex][\'gender\'] + \'_\' + \r\n                      tidigitsarray[ex][\'speaker\'] + \'_\' + \r\n                      tidigitsarray[ex][\'digit\'] + \'_\' + \r\n                      tidigitsarray[ex][\'repetition\'])\r\n    return labels\r\n\r\ndef dither(samples, level=1.0):\r\n    """\r\n    Applies dithering to the samples. Adds Gaussian noise to the samples to avoid numerical\r\n        errors in the subsequent FFT calculations.\r\n\r\n        samples: array of speech samples\r\n        leve

In [0]:
from lab3_proto import *
from lab3_tools import *
from lab1_proto import *
import numpy as np
import tensorflow as tf
import keras
import pysndfile

In [24]:
phoneHMMs = np.load('/content/drive/My Drive/lab2_models_all.npz', allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]
print(stateList)

['ah_0', 'ah_1', 'ah_2', 'ao_0', 'ao_1', 'ao_2', 'ay_0', 'ay_1', 'ay_2', 'eh_0', 'eh_1', 'eh_2', 'ey_0', 'ey_1', 'ey_2', 'f_0', 'f_1', 'f_2', 'ih_0', 'ih_1', 'ih_2', 'iy_0', 'iy_1', 'iy_2', 'k_0', 'k_1', 'k_2', 'n_0', 'n_1', 'n_2', 'ow_0', 'ow_1', 'ow_2', 'r_0', 'r_1', 'r_2', 's_0', 's_1', 's_2', 'sil_0', 'sil_1', 'sil_2', 'sp_0', 't_0', 't_1', 't_2', 'th_0', 'th_1', 'th_2', 'uw_0', 'uw_1', 'uw_2', 'v_0', 'v_1', 'v_2', 'w_0', 'w_1', 'w_2', 'z_0', 'z_1', 'z_2']


In [29]:
filename = '/content/drive/My Drive/Data/train/boy/ss/z84a.wav'
samples, samplingrate = loadAudio(filename)
lmfcc = mfcc(samples)
print("LMFCC Shape: ", lmfcc.shape)

wordTrans = list(path2info(filename)[2])
print("Sequence of digits", wordTrans)

LMFCC Shape:  (271, 13)
['z', '8', '4']


In [0]:
# utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)